In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime, date, timedelta

import sqlite3
from sqlalchemy               import create_engine
from sqlalchemy.pool          import NullPool

# from sklearn.decomposition import PCA
# from umap.umap_ import UMAP
# from sklearn.manifold import TSNE

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
PATH = '../data/hotel_revenue_historical_full.xlsx'

In [4]:
df_raw_2018 = pd.read_excel(PATH, sheet_name='2018')
df_raw_2019 = pd.read_excel(PATH, sheet_name='2019')
df_raw_2020 = pd.read_excel(PATH, sheet_name='2020')
df_raw_meal_cost = pd.read_excel(PATH, sheet_name='meal_cost')
df_raw_market_segment = pd.read_excel(PATH, sheet_name='market_segment')

In [5]:
df_raw_2018.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'deposit_type', 'agent',
       'company', 'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date'],
      dtype='object')

In [15]:
hotels = pd.concat([df_raw_2018,df_raw_2019,df_raw_2020], axis=0).reset_index(drop = True)

In [16]:
hotels

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,1,85,2018,July,27,1,0,3,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,240.0,NaN,0,Transient,82.0,0,1,Canceled,2018-05-06
1,Resort Hotel,1,75,2018,July,27,1,0,3,2,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,D,D,0,No Deposit,15.0,NaN,0,Transient,105.5,0,0,Canceled,2018-04-22
2,Resort Hotel,1,23,2018,July,27,1,0,4,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,240.0,NaN,0,Transient,123.0,0,0,Canceled,2018-06-23
3,Resort Hotel,1,60,2018,July,27,1,2,5,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,E,E,0,No Deposit,240.0,NaN,0,Transient,107.0,0,2,Canceled,2018-05-11
4,Resort Hotel,1,96,2018,July,27,1,2,8,2,0.0,0,BB,PRT,Direct,Direct,0,0,0,E,E,0,No Deposit,NaN,NaN,0,Transient,108.3,0,2,Canceled,2018-05-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141942,City Hotel,1,1,2020,February,5,1,0,0,1,0.0,0,BB,AUT,Aviation,Corporate,1,0,1,A,A,0,No Deposit,NaN,153.0,0,Transient,0.0,0,1,No-Show,2020-02-01
141943,City Hotel,1,31,2020,July,29,16,2,0,1,0.0,0,BB,USA,Direct,Direct,1,0,2,A,A,1,No Deposit,NaN,NaN,0,Transient,135.0,0,2,No-Show,2020-07-16
141944,City Hotel,1,25,2020,May,18,6,2,1,1,0.0,0,BB,FRA,Corporate,Corporate,1,0,1,E,F,0,No Deposit,NaN,450.0,0,Transient,125.0,0,0,No-Show,2020-05-06
141945,City Hotel,1,6,2020,July,29,17,1,0,1,0.0,0,BB,PRT,Corporate,Corporate,1,1,1,A,D,0,No Deposit,NaN,238.0,0,Transient,65.0,0,0,No-Show,2020-07-17


In [17]:
df_raw_meal_cost

,Cost,meal
0,0.00,Undefined
1,12.99,BB
2,17.99,HB
3,21.99,FB
4,35.00,SC


In [33]:
#endpoint = 'sqlite:///insiders_db.sqlite' #local
endpoint = 'sqlite:////Users/Alysson/Documents/Projects/Hotel-Booking-Cancelation/data/hotels.sqlite' #local

db = create_engine(endpoint, poolclass=NullPool)
connection = db.connect()

In [34]:
connection = sqlite3.connect('/Users/Alysson/Documents/Projects/Hotel-Booking-Cancelation/data/hotels.sqlite')

In [35]:
#df_raw_2018

In [36]:
df_raw_market_segment.to_sql(
    name = 'market_segment',
    con = connection,
    if_exists = 'replace',
    index = False,
    dtype = {'Discount':'real',
             'market_segment':'text'        
            })

In [37]:
df_raw_meal_cost.to_sql(
    name = 'meal_cost',
    con = connection,
    if_exists = 'replace',
    index = False,
    dtype = {'Cost':'real',
             'meal':'text'        
            })

In [38]:
#df_raw_2018.columns

In [39]:
hotel_dtypes = {
            'hotel' : 'text' ,
            'is_canceled' : 'integer',
            'lead_time': 'integer',
            'arrival_date_year' : 'integer',
            'arrival_date_month': 'integer',
            'arrival_date_week_number' : 'integer',        
            'arrival_date_day_of_month' : 'integer',
            'stays_in_weekend_nights' : 'integer',
            'stays_in_week_nights' : 'integer',
            'adults' : 'integer',
            'children' : 'integer',
            'babies' :'integer',
            'meal' : 'text',        
            'country' : 'text',
            'market_segment' : 'text',
            'distribution_channel' : 'text',
            'is_repeated_guest' : 'integer',
            'previous_cancellations' : 'integer',        
            'previous_bookings_not_canceled' : 'integer',
            'reserved_room_type' : 'text',
            'assigned_room_type' : 'text',
            'booking_changes' : 'integer',
            'deposit_type' : 'text',
            'agent' : 'integer',
            'company': 'integer',
            'days_in_waiting_list' : 'integer',
            'customer_type' : 'text',
            'adr' : 'real',
            'required_car_parking_spaces' : 'integer',
            'total_of_special_requests' : 'integer',
            'reservation_status' : 'text',
            'reservation_status_date' : 'date'            
            }

In [40]:
# df_raw_2018.to_sql(
#     name = 'db_2018',
#     con = connection,
#     if_exists = 'replace',
#     index = False,
#     dtype = hotel_dtypes)

# df_raw_2019.to_sql(
#     name = 'db_2019',
#     con = connection,
#     if_exists = 'replace',
#     index = False,
#     dtype = hotel_dtypes

# df_raw_2020.to_sql(
#     name = 'db_2020',
#     con = connection,
#     if_exists = 'replace',
#     index = False,
#     dtype = hotel_dtypes)  

In [41]:
hotels.to_sql(
    name = 'hotels',
    con = connection,
    if_exists = 'replace',
    index = False,
    dtype = hotel_dtypes)

In [42]:
#check if table exists on sqlite
check_table = """
     SELECT name FROM sqlite_master WHERE type='db' AND name='hotels';
 """
df_check = pd.read_sql_query(check_table, connection)


In [43]:
len(df_check)

0

In [44]:
#0 = table does not exist, 1 = table exists
if len(df_check) == 0:
    print('Table loyals was created!')
else:
    print('Table loyals exists!')

Table loyals was created!


In [45]:
connection.commit()

In [47]:
connection.close()

In [ ]:
# #check if table exists on sqlite
# check_table = """
#      SELECT name FROM sqlite_master WHERE type='table' AND name='hotels';
#  """
# df_check = pd.read_sql_query(check_table, conn)

# #0 = table does not exist, 1 = table exists
# if len(df_check) == 0:  
#     query_create_table_insiders = """
#         CREATE TABLE insiders (
#             CustomerID               INTEGER,
#             GrossRevenue             REAL,
#             Recency                  INTEGER,
#             Products                 INTEGER,
#             Cluster                  INTEGER,
#             LastTraining             TEXT        
#            ) """


#     conn = sqlite3.connect('insiders_db_sqlite')
#     conn.execute( query_create_table_insiders )
#     print('Table loyals was created!')
# else:
#     print('Table loyals exists!')